Training pipeline:

In [ ]:
!pip install transformers[torch]
!pip install accelerate
!pip install datasets>=2.6.1
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer

Training and Evaluation

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

Load processed data from Google Drive

In [ ]:
from datasets import load_from_disk
common_voice["train"] = load_from_disk("/content/drive/MyDrive/common_voice/train") #.to_iterable_dataset()
common_voice["test"] = load_from_disk("/content/drive/MyDrive/common_voice/test") #.to_iterable_dataset()

Use some part of the test set

In [ ]:
common_voice

In [ ]:
num_rows_to_keep = int(len(common_voice["test"]) * 0.2)
common_voice["test"] = common_voice["test"].select(range(num_rows_to_keep))

In [ ]:
common_voice

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Turkish", task="transcribe")

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Turkish", task="transcribe")

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

==========

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Load a Pre-Trained Checkpoint

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
model.config.forced_decoder_ids = None
model.use_cache = False
model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/ID2223_Lab2_Whisper",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    save_strategy="no",
    do_eval = True,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=False,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

Save checkpoint

In [ ]:
from transformers import TrainerCallback
import os
class MyCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, model, optimizer, lr_scheduler, **kwargs):
      #checkpoint_dir = f"/content/drive/MyDrive/Whisper_Checkpoints/checkpoint-{state.global_step}"
      checkpoint_dir = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
      os.makedirs(checkpoint_dir, exist_ok=True)

      # Save model
      model.save_pretrained(checkpoint_dir, safe_serialization=True)

      # Save training arguments together with the trained model
      torch.save(args, os.path.join(checkpoint_dir, "training_args.bin"))

      # Save state
      state.save_to_json(os.path.join(checkpoint_dir, "trainer_state.json"))
      # Save optimizer
      torch.save(optimizer.state_dict(), os.path.join(checkpoint_dir, "optimizer.pt"))
      # Save scheduler
      torch.save(lr_scheduler.state_dict(), os.path.join(checkpoint_dir, "scheduler.pt"))

In [ ]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[MyCallback()],
)

In [ ]:
processor.save_pretrained(training_args.output_dir)

Load the saved checkpoint and train

In [ ]:
trainer.train(resume_from_checkpoint=True)
#trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: tr, split: test",
    "language": "tr",
    "model_name": "Whisper-Tr-id2223-lab2",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.push_to_hub(**kwargs)